In [1]:
#!/usr/bin/env python
# coding: utf-8

import os
import sys
import platform
import logging
import argparse
import trino
import io
import boto3
from itertools import islice
from datetime import datetime, date, timedelta
import pendulum
import pandas as pd
import numpy as np

In [2]:
# Add parent directory to path
repo_path = '/Users/apple/Documents/naarni/repo/dview-naarni-data-platform'
sys.path.append(os.path.join(repo_path, 'tasks'))

# Import necessary files and its respective functions
from common.db_operations import connect_to_trino, fetch_data_for_day, write_df_to_iceberg,drop_table,execute_query
from common.optimizer_logic import optimize_dataframe_memory

# Import business logic functions
from biz_logic.energy_mileage.energy_mileage_daily_v0 import energy_mileage_stats ,impute_odometer_readings

from biz_logic.energy_consumption.energy_consumption_report import energy_consumption_stats

# Configure basic logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')

# Print the Python version being used
print(f"Using Python version: {platform.python_version()}")

Using Python version: 3.13.7


In [3]:
# ---- reporting config (edit ONLY this) ----
TABLE_NAME = "can_parsed_output_100"   # <— change only this

# derived (don’t edit)
REPORT_TABLE = f"adhoc.facts_prod.{TABLE_NAME}"
REPORT_S3_LOCATION = f"s3a://naarni-data-lake/aqua/warehouse/facts_prod.db/{TABLE_NAME}/"

In [4]:
date_str = "2025-10-01"  # Date for which data is to be processed

# Parse the date string as a date object
target_date = datetime.strptime(date_str, "%Y-%m-%d").date()

# Create datetime objects for the start and end of the day in IST
ist_start = datetime.combine(target_date, datetime.min.time())
ist_end = ist_start + timedelta(days=1)

# Convert IST to UTC for the database query
# IST is UTC+5:30, so we subtract 5 hours and 30 minutes
utc_start = ist_start - timedelta(hours=5, minutes=30)
utc_end = ist_end - timedelta(hours=5, minutes=30)

logging.info(f"🔍 Query window (UTC): {utc_start} → {utc_end}")
logging.info(f"🔍 Query window (IST): {ist_start} → {ist_end}")

2025-10-27 18:38:31 - INFO - 🔍 Query window (UTC): 2025-09-30 18:30:00 → 2025-10-01 18:30:00
2025-10-27 18:38:31 - INFO - 🔍 Query window (IST): 2025-10-01 00:00:00 → 2025-10-02 00:00:00


In [ ]:
query = f"""
select 
    id,date(timestamp + interval '5:30' hour to minute) as dateval,count(*) 
FROM 
    {REPORT_TABLE} 
where 
    id in ('3','16','18') and 
    timestamp >= TIMESTAMP '{utc_start.strftime('%Y-%m-%d %H:%M:%S')}' and
    timestamp < TIMESTAMP '{utc_end.strftime('%Y-%m-%d %H:%M:%S')}'
group by 1,2"""

In [12]:
conn = connect_to_trino(host="analytics.internal.naarni.com",port=443,user="admin",catalog="adhoc",schema="default")

df = execute_query(conn, query , return_results=True)
display(df)

# df.to_csv("can_parsed_output_100_sample.csv", index=False)

2025-10-27 18:46:44 - INFO - 🔌 STEP 1: Connecting to Trino...
2025-10-27 18:46:44 - INFO - ✅ STEP 1: Connected to Trino
2025-10-27 18:46:44 - INFO - ⚙️ Executing query...
2025-10-27 18:47:05 - INFO - ✅ Query executed successfully with results!


,id,_col1,_col2
0,18,2025-10-01,65702
1,3,2025-10-01,59285
2,16,2025-10-01,74559
